In [1]:
from tokenizers import Tokenizer
from pathlib import Path
import sys

sys.path.append(str(Path(Path.cwd()).parent))

from helper.sentence_destructor import destruct_sentence, mask_sentence

### Test dataset destruction

In [2]:

tokenizer: Tokenizer = Tokenizer.from_file("../../config/input_tokenizer.json")

text = "مەن مەكتەپتىن قايتىپ كەلدىم، ئۇمۇ مەكتەپتىن قايتىپ كەلدى، بىز ھەممىمىز مەكتەپتىن قايتىپ كەلدۇق!"

for i in range(5):
    masked_text = mask_sentence(text, 0.2)
    destructed = destruct_sentence(tokenizer, text, 0.15)

    print(masked_text)
    print(destructed)
    print("-"*100)
print(text)

مەن مەكتەپتىن <MASK> كەلدىم، ئۇمۇ مەكتەپتىن قايتىپ كەلدى، بىز ھەممىمىز مەكتەپتىن <MASK> كەلدۇق!
مەن مەكتەپتىن قايىدپ كەلدى،م ئۇمۇ مەكتەپتخن قايتپ كەلتى، بز ھەممىمر مەۆتەپدىن قايتىپ كەلدۇق!
----------------------------------------------------------------------------------------------------
مەن مەكتەپتىن قايتىپ كەلدىم، ئۇمۇ مەكتەپتىن <MASK> كەلدى، بىز ھەممىمىز مەكتەپتىن قايتىپ كەلدۇق!
مەن مەكتەپتىن قايتىپ كەلدىم، ئۇمۇ مەكتەپتىن قايتىپ كەلدى، بىزھەممىمىز مەكتەپتىن قايتىپ كەلدۇق!
----------------------------------------------------------------------------------------------------
<MASK> مەكتەپتىن قايتىپ كەلدىم، ئۇمۇ مەكتەپتىن قايتىپ كەلدى، <MASK> <MASK> مەكتەپتىن قايتىپ كەلدۇق!
مەن مەكتەپتن قايتپ كەلدىم، ئۇمۇ مەكتەپتن قاتيپ كەلدى، بىز ھەمممز مكەتەپتن قايتىپ كەلدۇ!ق
----------------------------------------------------------------------------------------------------
مەن مەكتەپتىن قايتىپ كەلدىم، <MASK> <MASK> قايتىپ كەلدى، بىز <MASK> مەكتەپتىن قايتىپ كەلدۇق!
مەن مەكتەپتىن قايتپ كەلدم، ئۇمۇ مەك

### Read tsv files and write them to lmdb.

In [4]:
import csv
import pickle
from pathlib import Path
from tokenizers import Tokenizer, decoders, pre_tokenizers
import random
from tqdm import tqdm
import time
import lmdb

input_tokenizer: Tokenizer = Tokenizer.from_file(str(Path.cwd().parent.parent / "config/input_tokenizer.json"))
output_tokenizer: Tokenizer = Tokenizer.from_file(str(Path.cwd().parent.parent / "config/output_tokenizer.json"))
output_tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()
output_tokenizer.decoder = decoders.Metaspace()

#list all the tsv files in .data/text/by_sentences/*/tsv
tsv_files: list[Path] = list((Path.cwd().parent.parent / ".data/text/by_sentences").rglob("*.tsv"))

db = lmdb.Environment(str(Path.cwd().parent.parent / ".data/temp/by_sentences_lmdb"), map_size=1024 * 1024 * 1024 * 15)

id: int = 0

tx = db.begin(write=True)
def write_data(id: int, data: tuple[list[int], list[int]]):
    global db
    global tx
    tx.put(f"{id}".encode(), pickle.dumps(data))
    if id % 1000 == 0:
        tx.commit()
        tx = db.begin(write=True)

progress_bar = tqdm(tsv_files, desc="Processing tsv files", ncols=100)
for tsv_file in progress_bar:
    with open(tsv_file, "r") as file:
        reader = csv.reader(file, delimiter="\t")
        header = next(reader) #skip the header
        #find the "title_ug" index
        title_ug_index = header.index("title_ug")
        for row in reader:
            target_text = row[title_ug_index]
            target_ids: list[int] = output_tokenizer.encode(f"<SOS>{target_text}<EOS>").ids

            masked_target = mask_sentence(target_text, 0.2)
            misspelled_inputs = [destruct_sentence(input_tokenizer, target_text, random.uniform(0.1, 0.2)) for _ in range(3)]
            for misspelled_input in [masked_target] + misspelled_inputs:
                misspelled_input_ids: list[int] = input_tokenizer.encode(f"<SOS>{misspelled_input}<EOS>").ids
                if len(misspelled_input_ids) > 512:
                    continue
                pair = (misspelled_input_ids, target_ids)
                
                write_data(id, pair)
                id += 1
            
            #decrease the progress bar update frequency
                if id % 1000 == 0:
                    progress_bar.set_description(f"file: {tsv_file.name}, id: {id}")
tx.put(b"count", pickle.dumps(id))
tx.commit()

print(f"Added {id} pairs to SQLite database.")


file: sub_sentences_166.tsv, id: 16326000: 100%|██████████████████| 437/437 [35:17<00:00,  4.85s/it]

Added 16326748 pairs to SQLite database.
